In [ ]:
%load_ext autoreload
%autoreload 2
from aoc.lib import load, timing

YEAR = 2024
DAY = 8

In [ ]:
from collections import defaultdict
import numpy as np
from aoc.lib import CharGrid

@timing
def prepare_data():
    data = load(YEAR, DAY, split_lines=True)#, test='............\n........0...\n.....0......\n.......0....\n....0.......\n......A.....\n............\n............\n........A...\n.........A..\n............\n............\n')
    grid = CharGrid(data['split'])
    antennae = defaultdict(list)

    for pos in grid.walk():
        freq = grid[pos]
        if freq != '.':
            antennae[freq].append(np.array(pos))

    return grid, antennae

In [ ]:
# Level 1: Collect all "antinodes" created by mirroring same-frequency antennae on each other, and count them:
from itertools import combinations

def add_antinode(antinodes, grid, pos):
    pos = tuple(pos)
    if grid.is_inside(pos):
        antinodes.add(pos)

@timing
def level1(grid, antennae):
    antinodes = set()
    for freq, ant in antennae.items():
        for pos1, pos2 in combinations(ant, 2):
            add_antinode(antinodes, grid, 2 * pos1 - pos2)
            add_antinode(antinodes, grid, 2 * pos2 - pos1)
    return len(antinodes)


grid, antennae = prepare_data()
print(level1(grid, antennae))

In [ ]:
# Level 2: Same, but now antinodes are wherever the connection of two antennae falls directly on a grid point
# Approach: From the difference of each antennae pair find the smallest integer step, and use that to populate the antinode list
from math import gcd

def gen_antinodes(grid, a1, a2):
    # calc smallest integer step
    if a2[0] > a1[0]:
        diff = a2 - a1
    else:
        diff = a1 - a2
    div = gcd(diff[0], diff[1])
    diff = diff // div
    
    # go to first plausible column
    startx = a1[0] % diff[0]
    starty = a1[1] - (a1[0] - startx) * diff[1] // diff[0]
    a = np.array([startx, starty])
    
    # Move until inside:
    while not grid.is_inside(tuple(a)):
        a += diff
        
    # track antinodes:
    antinodes = set()
    while grid.is_inside(tuple(a)):
        antinodes.add(tuple(a))
        a += diff
        
    return antinodes


@timing
def level2(grid, antennae):
    antinodes = set()
    for freq, ant in antennae.items():
        for pos1, pos2 in combinations(ant, 2):
            antinodes = antinodes.union(gen_antinodes(grid, pos1, pos2))
    return len(antinodes)


grid, antennae = prepare_data()
print(level2(grid, antennae))